1. Install Required Packages

In [1]:
!pip install replicate duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.4 MB/s eta 0:00:00


2. Import Libraries and Set Up Clients

In [2]:
import replicate
from duckduckgo_search import DDGS

# Insert your Replicate API token here
REPLICATE_API_TOKEN = "r8_8gMwvz4Bl6gOW5hn0F229v1I3LRu6yz2Hn2tO"
client = replicate.Client(api_token=REPLICATE_API_TOKEN)


3. Optional: Search Function


In [3]:
def search_duckduckgo(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join([f"- {r['title']}: {r['href']}" for r in results])


4. Replicate Query Function

In [4]:
def query_replicate(prompt, model="ibm-granite/granite-3.3-8b-instruct"):
    output = client.run(model, input={"prompt": prompt, "max_new_tokens": 300})
    return "".join(output)


5. Define Agent Roles

a. Crop Advisor Agent

In [5]:
def crop_advisor_agent(prompt):
    system_prompt = "You are a smart crop advisor. Give crop selection and health advice based on soil, weather, and market info."
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


b. Weather & Irrigation Agent

In [6]:
def weather_irrigation_agent(prompt):
    system_prompt = "You are an AI monitoring weather and irrigation. Suggest schedules and warnings based on user location."
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


c. Pest and Disease Detection Agent

In [7]:
def pest_disease_agent(prompt):
    system_prompt = "You are an AI assistant specializing in crop pests and diseases. Provide quick diagnosis and treatment suggestions based on farm observations."
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


6. Routing User Queries

In [8]:
def smart_farming_router(user_prompt):
    crop_keywords = ["crop", "plant", "harvest", "yield", "seed"]
    weather_keywords = ["weather", "rain", "temperature", "irrigation", "moisture", "water"]
    pest_keywords = ["pest", "disease", "fungus", "insect", "infestation"]

    prompt_lower = user_prompt.lower()

    if any(word in prompt_lower for word in crop_keywords):
        return crop_advisor_agent(user_prompt)
    elif any(word in prompt_lower for word in weather_keywords):
        return weather_irrigation_agent(user_prompt)
    elif any(word in prompt_lower for word in pest_keywords):
        return pest_disease_agent(user_prompt)
    else:
        return "🌱 Please specify if you want crop advice, weather/irrigation help, or pest/disease diagnosis."


7. Gradio App Interface

In [9]:
import gradio as gr

def gradio_interface(user_prompt):
    return smart_farming_router(user_prompt)

app = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, label="Ask about crops, weather, or pests…"),
    outputs=gr.Textbox(label="Smart Farming AI Response"),
    title="Smart Farming AI Agent",
    description="This AI agent gives smart advice on crops, weather/irrigation, and pest/disease management. Enter any farm-related question!"
)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce5209c4ce154c4cb9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
